# Demonstration of the topic coherence pipeline in Gensim

## Introduction

We will be using the `c_v` coherence for two different LDA models: a "good" and a "bad" LDA model. The good LDA model will be trained over 50 iterations and the bad one for 1 iteration. Hence in theory, the good LDA model will be able come up with better or more human-understandable topics. Therefore the coherence measure output for the good LDA model should be more (better) than that for the bad LDA model. This is because, simply, the good LDA model usually comes up with better topics that are more human interpretable.

In [1]:
from __future__ import print_function

import logging
import warnings

from gensim.models import CoherenceModel, LdaModel
from gensim.corpora import Dictionary

warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

import load_lee_background_corpus as load_texts

### Set up corpus

In [2]:
texts = load_texts.get_train_texts()

/Users/home/Desenvolvimento/anaconda3/lib/python3.6/site-packages/gensim/test/test_data


2017-11-22 13:57:30,794 : INFO : collecting all words and their counts
2017-11-22 13:57:30,796 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2017-11-22 13:57:30,846 : INFO : collected 20429 word types from a corpus of 19878 words (unigram + bigrams) and 300 sentences
2017-11-22 13:57:30,847 : INFO : using 20429 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


Returning 300 training texts


In [3]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2017-11-22 13:57:35,193 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-11-22 13:57:35,239 : INFO : built Dictionary(3824 unique tokens: ['hundred', 'people', 'home', 'southern', 'highland']...) from 300 documents (total 18861 corpus positions)


### Set up two topic models

We'll be setting up two different LDA Topic models. A good one and bad one. To build a "good" topic model, we'll simply train it using more iterations than the bad one. Therefore the coherence should in theory be better for the good model than the bad one since it would be producing more "human-interpretable" topics.

In [4]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=200, passes=5, num_topics=10)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=10)

2017-11-22 13:58:32,114 : INFO : using symmetric alpha at 0.1
2017-11-22 13:58:32,117 : INFO : using symmetric eta at 0.0002615062761506276
2017-11-22 13:58:32,120 : INFO : using serial LDA version on this node
2017-11-22 13:58:32,572 : INFO : running online (multi-pass) LDA training, 10 topics, 5 passes over the supplied corpus of 300 documents, updating model once every 300 documents, evaluating perplexity every 300 documents, iterating 200x with a convergence threshold of 0.001000
2017-11-22 13:58:32,574 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2017-11-22 13:58:33,174 : DEBUG : bound: at document #0
2017-11-22 13:58:36,314 : INFO : -11.109 per-word bound, 2209.1 perplexity estimate based on a held-out corpus of 300 documents with 18861 words
2017-11-22 13:58:36,316 : INFO : PROGRESS: pass 0, at document #300/300
2017-11-22 13:58:36,318 : DEBUG : performing inference on a chunk of 300 document

2017-11-22 13:58:47,515 : INFO : topic #8 (0.100): 0.015*"people" + 0.010*"year" + 0.008*"report" + 0.008*"new_york" + 0.007*"australia" + 0.006*"city" + 0.006*"economy" + 0.006*"rate" + 0.005*"world" + 0.005*"government"
2017-11-22 13:58:47,518 : INFO : topic #9 (0.100): 0.013*"arafat" + 0.010*"australia" + 0.009*"qantas" + 0.008*"people" + 0.008*"attack" + 0.007*"test" + 0.007*"day" + 0.007*"government" + 0.007*"group" + 0.007*"official"
2017-11-22 13:58:47,521 : INFO : topic #7 (0.100): 0.015*"israeli" + 0.012*"arafat" + 0.011*"metre" + 0.009*"security" + 0.009*"palestinian" + 0.008*"israel" + 0.007*"force" + 0.007*"west_bank" + 0.007*"militant" + 0.007*"official"
2017-11-22 13:58:47,524 : INFO : topic diff=0.090895, rho=0.408248
2017-11-22 13:58:47,528 : INFO : using symmetric alpha at 0.1
2017-11-22 13:58:47,531 : INFO : using symmetric eta at 0.0002615062761506276
2017-11-22 13:58:47,534 : INFO : using serial LDA version on this node
2017-11-22 13:58:48,031 : INFO : running onlin

### View the pipeline parameters for one coherence model

In [5]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
badcm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')

2017-11-22 13:59:36,510 : DEBUG : Setting topics to those of the model: LdaModel(num_terms=3824, num_topics=10, decay=0.5, chunksize=2000)
2017-11-22 13:59:36,515 : DEBUG : Setting topics to those of the model: LdaModel(num_terms=3824, num_topics=10, decay=0.5, chunksize=2000)


### Pipeline parameters for C_V coherence

In [6]:
print(goodcm)

Coherence_Measure(seg=<function s_one_set at 0x1a076a3e18>, prob=<function p_boolean_sliding_window at 0x1a07696d08>, conf=<function cosine_similarity at 0x1a076fde18>, aggr=<function arithmetic_mean at 0x1a076ff400>)


### Print coherence values

In [7]:
print(goodcm.get_coherence())
print(badcm.get_coherence())

2017-11-22 14:00:42,194 : INFO : using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
2017-11-22 14:00:42,342 : DEBUG : completed batch 0; 64 documents processed (92 virtual)
2017-11-22 14:00:42,398 : DEBUG : completed batch 0; 64 documents processed (258 virtual)
2017-11-22 14:00:42,422 : DEBUG : completed batch 0; 64 documents processed (305 virtual)
2017-11-22 14:00:42,431 : DEBUG : observed sentinel value; terminating
2017-11-22 14:00:42,434 : DEBUG : finished all batches; 64 documents processed (305 virtual)
2017-11-22 14:00:42,438 : INFO : serializing accumulator to return to master...
2017-11-22 14:00:42,442 : INFO : accumulator serialized
2017-11-22 14:00:42,503 : DEBUG : completed batch 1; 128 documents processed (332 virtual)
2017-11-22 14:00:42,506 : DEBUG : observed sentinel value; terminating
2017-11-22 14:00:42,509 : DEBUG : finished all batches; 128 documents processed (332 virtual)
2017-11-22 14:00:42,512 : I

0.350333107184


2017-11-22 14:00:43,446 : DEBUG : completed batch 0; 64 documents processed (92 virtual)
2017-11-22 14:00:43,498 : DEBUG : completed batch 0; 64 documents processed (305 virtual)
2017-11-22 14:00:43,496 : DEBUG : completed batch 0; 64 documents processed (258 virtual)
2017-11-22 14:00:43,503 : DEBUG : observed sentinel value; terminating
2017-11-22 14:00:43,506 : DEBUG : finished all batches; 64 documents processed (305 virtual)
2017-11-22 14:00:43,511 : INFO : serializing accumulator to return to master...
2017-11-22 14:00:43,501 : DEBUG : completed batch 1; 128 documents processed (332 virtual)
2017-11-22 14:00:43,514 : INFO : accumulator serialized
2017-11-22 14:00:43,535 : DEBUG : observed sentinel value; terminating
2017-11-22 14:00:43,538 : DEBUG : finished all batches; 128 documents processed (332 virtual)
2017-11-22 14:00:43,541 : INFO : serializing accumulator to return to master...
2017-11-22 14:00:43,545 : INFO : accumulator serialized
2017-11-22 14:00:43,577 : DEBUG : compl

0.272100427948
